# AutoBrochure
To develop an AI-powered system capable of intelligently scraping, understanding, and synthesizing company website content to automatically generate structured, high-quality brochures.

In [2]:
# Import Required Libraries
import requests
from bs4 import BeautifulSoup
from dotenv import load_dotenv
import os
from IPython.display import Markdown, display, update_display
from openai import OpenAI
import json

In [3]:
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
openai = OpenAI()


In [4]:
# Standard headers to fetch a website
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

In [5]:
def fetch_website_links(url):
    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.content,"html.parser")
    links =[link.get("href") for link in soup.find_all("a")]
    return [link for link in links if link]
    
def fetch_website_contents(url):
    """Fetch and return title and first 2000 characters of text from a website"""
    response =requests.get(url)
    soup = BeautifulSoup(response.text,"html.parser")

    title = soup.title.string if soup.title else "No title found"
    # Remove unnecesscary tags
    for tag in soup(["script","style","img","input"]):
        tag.decompose()
    text = soup.body.get_text(separator="\n", strip=True)
    return(f"{title} \n\n {text}")

In [6]:
links =fetch_website_links("https://www.ikea.com")
links

['#main',
 'https://www.ikea.com',
 'https://www.ikea.com/global/en/stories/',
 'https://www.ikea.com/global/en/jobs/',
 'https://www.ikea.com/global/en/newsroom/',
 'https://www.ikea.com/global/en/our-business/',
 'https://www.ikea.com/global/en/stories/',
 'https://www.ikea.com/global/en/jobs/',
 'https://www.ikea.com/global/en/newsroom/',
 'https://www.ikea.com/global/en/our-business/',
 'https://www.ikea.com/global/en/stories/our-roots/meatball-40-years-251024/',
 'https://www.ikea.com/global/en/stories/design/beautiful-bin-movement-251105/',
 'https://www.ikea.com/global/en/stories/ideas-inspiration/light-up-your-home-251029/',
 'https://www.ikea.com/global/en/stories/design/sandlopare-collection-251001/',
 'https://www.ikea.com/global/en/stories/design/gustaf-westman-ikea-collection-250909/',
 'https://www.ikea.com/global/en/stories/design/ommjange-collection-251008/',
 'https://www.ikea.com/global/en/stories/ideas-inspiration/kitchen-matchmaker/250903/',
 'https://www.ikea.com/g

In [7]:
## Figure if Link is relevant using LLM
link_system_prompt = """
You are provided with a list of links found on a webpage.
You are able to decide which of the links would be most relevant to include in a brochure about the company,
such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:

{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page", "url": "https://another.full.url/careers"}
    ]
}
"""

In [8]:
def get_links_user_prompt(url):
    user_prompt = f"""
                    Here is the list of links on the website {url} -
                    Please decide which of these are relevant web links for a brochure about the company, 
                    respond with the full https URL in JSON format.
                    Do not include Terms of Service, Privacy, email links.
                    
                    Links (some might be relative links):
                    
                    """
    links = fetch_website_links(url)
    user_prompt +="\n".join(links)
    return user_prompt

In [9]:
print(get_links_user_prompt("https://www.ikea.com"))


                    Here is the list of links on the website https://www.ikea.com -
                    Please decide which of these are relevant web links for a brochure about the company, 
                    respond with the full https URL in JSON format.
                    Do not include Terms of Service, Privacy, email links.
                    
                    Links (some might be relative links):
                    
                    #main
https://www.ikea.com
https://www.ikea.com/global/en/stories/
https://www.ikea.com/global/en/jobs/
https://www.ikea.com/global/en/newsroom/
https://www.ikea.com/global/en/our-business/
https://www.ikea.com/global/en/stories/
https://www.ikea.com/global/en/jobs/
https://www.ikea.com/global/en/newsroom/
https://www.ikea.com/global/en/our-business/
https://www.ikea.com/global/en/stories/our-roots/meatball-40-years-251024/
https://www.ikea.com/global/en/stories/design/beautiful-bin-movement-251105/
https://www.ikea.com/global/en/stories/i

In [10]:
def select_relevant_links(url):
    response =openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role":"system","content":link_system_prompt},
            {"role":"user","content":get_links_user_prompt(url)}
        ],
        response_format={"type":"json_object"}
    )
    result =response.choices[0].message.content
    links = json.loads(result)
    return links

In [11]:
print(select_relevant_links("https://www.ikea.com"))

{'links': [{'type': 'about page', 'url': 'https://www.ikea.com/global/en/our-business/'}, {'type': 'careers page', 'url': 'https://www.ikea.com/global/en/jobs/'}, {'type': 'newsroom page', 'url': 'https://www.ikea.com/global/en/newsroom/'}, {'type': 'stories page', 'url': 'https://www.ikea.com/global/en/stories/'}, {'type': 'foundation page', 'url': 'https://ikeafoundation.org/'}, {'type': 'museum page', 'url': 'https://ikeamuseum.com/en/'}, {'type': 'social entrepreneurship page', 'url': 'https://www.ikeasocialentrepreneurship.org/'}]}


In [12]:
## Create the brochure
def fetch_page_and_all_relevant_links(url):
    links = select_relevant_links(url)
    result =""
    for link in links["links"]:
        result += f"\n\n### Link: {link['type']}\n"
        result += fetch_website_contents(link['url'])
    
    return result

In [13]:
print(fetch_page_and_all_relevant_links("https://www.ikea.com"))



### Link: about page
Our business - IKEA Global 

 Skip to main content
Stories
Jobs
Newsroom
Our business
Open main menu
Close main menu
Stories
Jobs
Newsroom
Our business
Our business
How we work
Sustainability
Our view on
Reports
International Sales
•
Inter IKEA Group
•
IKEA Foundation
•
IKEA Museum
•
IKEA Social Entrepreneurship
•
FAQs
Cookie settings
•
Privacy & Cookie statement
•
Contact IKEA / Submit complaint
•
Sitemap
•
Accessibility
© Inter IKEA Systems B.V. 1999 - 2025

### Link: careers page
Jobs: The key to your future – IKEA Global 

 Skip to main content
Stories
Jobs
Newsroom
Our business
Open main menu
Close main menu
Stories
Jobs
Newsroom
Our business
Jobs
The key to your future
Do you dream of helping to create a better everyday life for the many people? You’re in the right place!
Play video of a revolving Allen key.
Whether you're just starting out in a new career, or simply looking for your next challenge, looking for a new job can be as daunting as assembling you

In [14]:
brochure_system_prompt = """
You are an assistant that analyzes the contents of several relevant pages from a company website
and creates a short brochure about the company for prospective customers, investors and recruits.
Respond in markdown without code blocks.
Include details of company culture, customers and careers/jobs if you have the information.
"""

In [15]:
def get_brochure_user_prompt(company_name,url):
    user_prompt = f"""
                    You are looking at a company called: {company_name}
                    Here are the contents of its landing page and other relevant pages;
                    use this information to build a short brochure of the company in markdown without code blocks.\n\n
                    """
    user_prompt += fetch_page_and_all_relevant_links(url)
    user_prompt =user_prompt[:5000]
    return user_prompt

In [16]:
print(get_brochure_user_prompt("Ikea", "https://www.ikea.com"))



                    You are looking at a company called: Ikea
                    Here are the contents of its landing page and other relevant pages;
                    use this information to build a short brochure of the company in markdown without code blocks.


                    

### Link: about page
Our business - IKEA Global 

 Skip to main content
Stories
Jobs
Newsroom
Our business
Open main menu
Close main menu
Stories
Jobs
Newsroom
Our business
Our business
How we work
Sustainability
Our view on
Reports
International Sales
•
Inter IKEA Group
•
IKEA Foundation
•
IKEA Museum
•
IKEA Social Entrepreneurship
•
FAQs
Cookie settings
•
Privacy & Cookie statement
•
Contact IKEA / Submit complaint
•
Sitemap
•
Accessibility
© Inter IKEA Systems B.V. 1999 - 2025

### Link: careers page
Jobs: The key to your future – IKEA Global 

 Skip to main content
Stories
Jobs
Newsroom
Our business
Open main menu
Close main menu
Stories
Jobs
Newsroom
Our business
Jobs
The key to your future
Do yo

In [17]:
def create_brochure(company_name,url):
    response =openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role":"system","content":brochure_system_prompt},
            {"role":"user","content":get_brochure_user_prompt(company_name,url)}
        ]
    )
    result =response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("Ikea", "https://www.ikea.com")

# IKEA Company Brochure

---

## About IKEA

IKEA is a global leader in home furnishings, deeply rooted in the forests of southern Sweden where it was founded over 80 years ago in the town of Älmhult. Today, IKEA operates worldwide through a franchise system with 12 franchisee groups across 63 markets, bringing affordable, well-designed, and sustainable products to millions of customers.

With a vast “value chain” comprising over 200,000 co-workers, IKEA’s business includes the imagining, prototyping, production, transportation, management, marketing, and retailing of home furnishings. The brand also offers culinary experiences through IKEA Food.

IKEA’s mission goes beyond retail — the company strives to make a positive impact on the world, from sustainable sourcing of materials to helping customers live more sustainably at home.

---

## Company Culture & Values

- **One Big ‘We’**: IKEA champions a strong, inclusive culture where everyone’s ideas and unique talents are welcomed. Cooperation and mutual support are at the heart of working life.
- **Always Evolving**: IKEA embraces change and innovation. Employees are encouraged to question the status quo and think differently to improve how they work and serve customers better.
- **Diversity and Inclusion**: Equality is a cornerstone—with fair treatment and equal opportunities for all regardless of gender, age, ethnicity, disability, religion, or any other identity aspect.
- **Balance & Wellbeing**: IKEA cultivates a playful, supportive environment that promotes healthy everyday choices and well-being for its co-workers around the globe.
- **Swedish Roots, Global Spirit**: While IKEA’s origins remain central in Älmhult, its spirit and impact are truly global.

---

## Careers at IKEA

### Your Future at IKEA

Whether you’re starting your career, seeking a new challenge, or exploring a different direction, IKEA offers a wide range of opportunities across stores, development centres, factories, distribution facilities, studios, and offices worldwide.

- Join a network of **200,000+ global co-workers** in diverse roles: from creating and producing products, managing logistics, marketing, or customer service, to digital innovation such as drone deliveries.
- Enjoy **career growth and mobility**—many co-workers grow by moving between roles and countries.
- Find a workplace that encourages creativity, individuality, and collaboration.
- Benefit from a supportive environment focused on **personal and professional development**, wellbeing, and work-life balance.

### Life at IKEA

IKEA is more than a job—it's a community where voices are heard, and co-workers come together to constantly improve and innovate. Team members like Amplification Manager Antonia Banuelos-Leon express that change and diversity in tasks are key drivers of motivation and satisfaction.

### Join the IKEA Family

If you believe in creating a better everyday life for the many people and want to contribute to a sustainable future, IKEA could be the key to your next opportunity.

Explore current openings and apply: **[IKEA Careers](https://www.ikea.com)**

---

## Customers & Sustainability

IKEA aims to serve the many people worldwide by providing products that are accessible, sustainable, and designed to improve daily living. The company embeds sustainability in its business strategy — from responsible sourcing to enabling customers to live more eco-friendly lives.

---

## Connect with IKEA

Follow IKEA’s latest stories, innovations, and career opportunities by engaging on their global platforms. Whether you want to stay informed, apply for jobs, or join the IKEA community, they welcome you to be part of their journey.

---

**IKEA**  
Better Living for the Many People  
Since 1943, Älmhult, Sweden — Now Worldwide

---

In [19]:
def stream_brochure(company_name,url):
    stream =openai.chat.completions.create(
        model="gpt-4.1-mini",
        messages=[
            {"role":"system","content":brochure_system_prompt},
            {"role":"user","content":get_brochure_user_prompt(company_name,url)}
        ],
        stream =True
    )
    response =""
    display_handle =display(Markdown(""),display_id=True)
    for chunk in stream:
        response +=chunk.choices[0].delta.content or ''
        update_display(Markdown(response),display_id=display_handle.display_id)

In [20]:
stream_brochure("Airbnb", "https://www.airbnb.com")

# Airbnb: Belong Anywhere

---

## About Airbnb

Airbnb is a global travel community that connects millions of people looking for unique accommodations and experiences around the world. Founded with the mission to create a world where anyone can belong anywhere, Airbnb revolutionizes the way people travel by offering access to millions of homes, apartments, and unique stays in over 220 countries and regions.

---

## Our Culture

At Airbnb, we believe in promoting a diverse, inclusive, and supportive work environment where creativity and collaboration thrive. Our culture is built on the core values of belonging, trust, and empowerment, encouraging our team members to innovate and make a positive impact on the global travel community. We emphasize inclusivity, sustainability, and social impact in everything we do, fostering a workplace where everyone can grow both personally and professionally.

---

## Customers

Airbnb serves a wide variety of customers, including:

- Travelers seeking authentic local experiences in homes, boutique hotels, or unique stays worldwide.
- Hosts who want to share their space and culture while earning extra income.
- Businesses looking for flexible and inspiring travel accommodations.
- Communities and neighborhoods that benefit from the economic opportunity and cultural exchange that Airbnb facilitates.

Our platform is designed to foster meaningful connections, creating trust between guests and hosts through verified profiles, secure payments, and transparent reviews.

---

## Careers & Opportunities

Airbnb offers exciting careers for individuals passionate about transforming the future of travel and hospitality. We seek talented professionals in fields including technology, design, marketing, customer experience, and business development.

By joining Airbnb, you become part of a mission-driven company that values innovation, diversity, and social responsibility. We provide a dynamic and supportive workplace where employees can learn, grow, and contribute to creating global belonging.

---

## Why Choose Airbnb?

- **Unique Travel Experiences:** Access to millions of unique stays and activities worldwide.
- **Community Focus:** A platform built on trust, inclusivity, and shared experiences.
- **Global Reach:** Presence in over 220 countries and regions.
- **Sustainable Travel:** Commitment to responsible hosting and community engagement.
- **Innovative Technology:** Seamless and secure platform facilitating easy booking and hosting.

---

Experience the world differently — **Belong Anywhere** with Airbnb.

---